# Análisis final

En esta notebook, analicemos con los datos etiquetados de la academia cómo quedó cada métrica.

Para ello, integraremos los datos del último etiquetado de la Academia, que es el localizado en `data/listado_definitivo_2.csv`


In [1]:
%pylab inline
%load_ext autoreload
%autoreload 2

import pandas as pd
import contrastes

df = contrastes.read_occurrence_dataframe("output/listados/listado_completo.csv")
nuevas_etiquetadas = pd.read_csv("data/listado_definitivo_2.csv")

nuevas_etiquetadas.set_index("palabra", inplace=True)


Populating the interactive namespace from numpy and matplotlib


/home/jmperez/projects/contrastes/notebooks/contrastes/processing.py:147: UserWarning: Pandas doesn't allow columns to be created via a new attribute name - see https://pandas.pydata.org/pandas-docs/stable/indexing.html#attribute-access
  df.cant_palabras = cant_palabras
/home/jmperez/projects/contrastes/notebooks/contrastes/processing.py:148: UserWarning: Pandas doesn't allow columns to be created via a new attribute name - see https://pandas.pydata.org/pandas-docs/stable/indexing.html#attribute-access
  df.cant_personas = cant_personas


Las que sean 0.5, las pongo como uno...

In [2]:
df.loc[df.etiqueta == 0.5, "etiqueta"] = 1.0

In [3]:
print("Cantidad total de palabras: {}".format(df.shape[0]))
print("Cantidad de palabras etiquetadas: {}".format(df.etiqueta.notna().sum()))

Cantidad total de palabras: 136847
Cantidad de palabras etiquetadas: 4853


Ahora, veamos sólo las 1000 primeras de cada métrica, que son las que nos importan

In [4]:
listado_total = df[(df.rank_palabras <= 1000) | (df.rank_personas <= 1000) | (df.rank_palper <= 1000)].copy()

listado_total.shape

(1380, 108)

Ok, 1380. ¿Cuántas quedan sin etiquetar?

In [5]:
faltantes = listado_total[listado_total.etiqueta.isna()]

print("Faltan etiquetar {} palabras".format(faltantes.shape[0]))
print("De estas, {} son lugares".format(sum(faltantes.es_lugar)))

Faltan etiquetar 460 palabras
De estas, 22 son lugares


¿Y cuantas tenemos en el nuevo etiquetado?

In [6]:
nuevas_etiquetadas.shape

(460, 4)

¿Están todas?

In [7]:
faltantes.index.difference(nuevas_etiquetadas.index)

Index([], dtype='object', name='palabra')

Así parece, están todas, iupi!

In [8]:
df.loc[faltantes.index, "etiqueta"] = nuevas_etiquetadas["candidata"].astype("float64")
listado_total.loc[faltantes.index, "etiqueta"] = nuevas_etiquetadas["candidata"].astype("float64")

In [9]:
listado_palabras = listado_total[listado_total.rank_palabras <= 1000]
listado_personas = listado_total[listado_total.rank_personas <= 1000]
listado_palper = listado_total[listado_total.rank_palper <= 1000]
listado_palabras.shape, listado_personas.shape, listado_palper.shape

((1000, 108), (1000, 108), (1000, 108))

In [10]:
p_palabras = listado_palabras.etiqueta.mean()
p_personas = listado_personas.etiqueta.mean()
p_palper = listado_palper.etiqueta.mean()

print("Porcentaje palabras          = {:.2f}%".format(p_palabras*100))
print("Porcentaje personas          = {:.2f}%".format(p_personas*100))
print("Porcentaje palabras-personas = {:.2f}%".format(p_palper*100))


Porcentaje palabras          = 21.90%
Porcentaje personas          = 30.20%
Porcentaje palabras-personas = 25.30%


La mejor métrica para esto quedó en contar personas

In [25]:
etiquetadas = df[df.etiqueta.notna()]

(etiquetadas.rank_personas <= 5000).sum()

2548

**Pregunta**: Me genera confusión es esto: ¿por qué quedaron sólo 2.5K etiquetadas de las 5000 primeras? :-\

**Respuesta**: Es que simplificamos bastante la métrica!